In [7]:
from pathlib import Path
import polars as pl
import polars.selectors as cs
import pandas as pd

In [4]:
OUTDIR = "/project/def-nahee/kbas/POM_Response_Parquet"
TRAIN_DIR = Path(OUTDIR) / "merged" / "train"

In [5]:
tables = sorted([p.name for p in TRAIN_DIR.iterdir() if p.is_dir()])
print("tables in train:", tables)

FileNotFoundError: [Errno 2] No such file or directory: '/project/def-nahee/kbas/POM_Response_Parquet/merged/train'

In [6]:
FEATURE_TABLE = "features" 
FEATURE_DIR = TRAIN_DIR / FEATURE_TABLE
print("feature dir:", FEATURE_DIR)

feature dir: /project/def-nahee/kbas/POM_Response_Parquet/merged/train/features


In [ ]:
lf = pl.scan_parquet(str(FEATURE_DIR / "*.parquet"))

q = (
    lf.select([
        cs.numeric().quantile(0.25).suffix("_p25"),
        cs.numeric().quantile(0.50).suffix("_p50"),
        cs.numeric().quantile(0.75).suffix("_p75"),
    ])
    .collect()
)

row = q.to_dicts()[0]
features = sorted({k.rsplit("_", 1)[0] for k in row.keys()})

out = pd.DataFrame({
    "feature": features,
    "p25": [row[f"{f}_p25"] for f in features],
    "p50": [row[f"{f}_p50"] for f in features],
    "p75": [row[f"{f}_p75"] for f in features],
})

In [ ]:
# out_path = Path(OUTDIR) / "train_feature_percentiles.csv"
# out.to_csv(out_path, index=False)
# print("saved:", out_path)
# out.head()